In [15]:
print("Ok")

Ok


In [16]:
%pwd

'c:\\Users\\vinee\\OneDrive\\Desktop\\Med-Chatbot\\Medical-AI-Assistant'

In [17]:
import os
os.chdir("../")

In [18]:
%pwd

'c:\\Users\\vinee\\OneDrive\\Desktop\\Med-Chatbot'

In [19]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [1]:
#Extract Data From the PDF File

def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents


In [2]:
extracted_data

NameError: name 'extracted_data' is not defined

In [11]:
#Splitting the data into text chunks

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [12]:
#checking
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 39994


In [23]:
#text_chunks
from langchain.embeddings import HuggingFaceEmbeddings

In [29]:
#Downloading the Embeddings from HuggingFace

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [30]:
embeddings = download_hugging_face_embeddings()

c:\Users\vinee\anaconda3\envs\Healix\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vinee\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [33]:
query_result = embeddings.embed_query("Hellow world")
print("Length", len(query_result))

Length 384


In [36]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_6zhZoB_BwKU1neaWwyaRa1NpamZBdKJ6hJ7phoQf2tLrDe2tkD1DNk5XaNhi1YGfH5ZHd2")

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [60]:
import os
# os.environ["PINECONE_API_KEY"] = "pcsk_6zhZoB_BwKU1neaWwyaRa1NpamZBdKJ6hJ7phoQf2tLrDe2tkD1DNk5XaNhi1YGfH5ZHd2"
# os.environ["GROQ_API_KEY"] = "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"

from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [43]:
#Embedding each chunk and inserting into the Pinecone index
from langchain_community.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [44]:
#Loading the existing index
from langchain_community.vectorstores import Pinecone

docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding = embeddings
)

In [45]:
docsearch

In [46]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [47]:
retrieved_docs = retriever.invoke("What is Cancer?")

In [48]:
retrieved_docs

[Document(metadata={'page': 730.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Cancer\nDefinition\nCancer is not just one disease, but a large group of\nalmost 100 diseases. Its two main characteristics are\nuncontrolled growth of the cells in the human body\nand the ability of these cells to migrate from the\noriginal site and spread to distant sites. If the spread\nis not controlled, cancer can result indeath.\nDescription\nOne out of every four deaths in the United States\nis from cancer. It is second only to heart disease as a\ncause of death in the states. About 1.2 million'),
 Document(metadata={'page': 671.0, 'source': 'Data\\Medical_book.pdf'}, page_content='Definition\nBreast cancer is caused by the development of\nmalignant cells in the breast. The malignant cells\noriginate in the lining of the milk glands or ducts of\nthe breast (ductal epithelium), defining this malig-\nnancy as a cancer. Cancer cells are characterized by\nuncontrolled division leading to abnormal g

In [69]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0.4, max_tokens=500)

In [71]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0.4, max_tokens=500)

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer "
    "the question. If you dont know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

In [72]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [84]:
response = rag_chain.invoke({"input": "what is Klinefelter syndrome ?"})
print(response["answer"])

Klinefelter syndrome is a chromosomal disorder that affects only males, causing learning disabilities, behavior problems, and an increased risk for certain health issues. It is one of the most common chromosomal abnormalities, affecting about 1 in every 500 to 800 males. You can find more information and resources from organizations such as Klinefelter's Organization, Klinefelter Syndrome and Associates, and the National Organization for Rare Disorders.
